In [27]:
import torch
import torch.nn as nn


class VGG11BackBone(nn.Module):
  def __init__(self):
    super(VGG11BackBone, self).__init__()

    self.relu = nn.ReLU(inplace=True)

    # Convolution Feature Extraction Part
    self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
    self.bn1 = nn.BatchNorm2d(64)
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

    self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
    self.bn2 = nn.BatchNorm2d(128)
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

    self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
    self.bn3_1 = nn.BatchNorm2d(256)
    self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
    self.bn3_2 = nn.BatchNorm2d(256)
    self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

    self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
    self.bn4_1 = nn.BatchNorm2d(512)
    self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.bn4_2 = nn.BatchNorm2d(512)
    self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

    self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.bn5_1 = nn.BatchNorm2d(512)
    self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.bn5_2 = nn.BatchNorm2d(512)

  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.pool1(x)

    x = self.conv2(x)
    x = self.bn2(x)
    x = self.relu(x)
    x = self.pool2(x)

    x = self.conv3_1(x)
    x = self.bn3_1(x)
    x = self.relu(x)
    x = self.conv3_2(x)
    x = self.bn3_2(x)
    x = self.relu(x)
    x = self.pool3(x)

    x = self.conv4_1(x)
    x = self.bn4_1(x)
    x = self.relu(x)
    x = self.conv4_2(x)
    x = self.bn4_2(x)
    x = self.relu(x)
    x = self.pool4(x)

    x = self.conv5_1(x)
    x = self.bn5_1(x)
    x = self.relu(x)
    x = self.conv5_2(x)
    x = self.bn5_2(x)
    x = self.relu(x)

    return x


In [28]:
class VGG11Classification(nn.Module):
  def __init__(self,num_classes=7):
    super(VGG11Classification,self).__init__()
    self.backbone=VGG11BackBone()
    self.pool5=nn.MaxPool2d(kernel_size=2, stride=2)
    self.gap=nn.AdaptiveAvgPool2d(1)
    self.fc_out=nn.Linear(512,num_classes)

  def forward(self,x):
    x=self.backbone(x)
    x=self.pool5(x)
    x=self.gap(x)
    x=torch.flatten(x)
    x=self.fc_out(x)
    return x

In [29]:
class VGG11Segmentation(nn.Module):
  def __init(self,num_classes=7):
    super(VGG11Segmentation,self).__init__()
    self.backbone=VGG11BackBone()
    self.conv_out=nn.Conv2d(512,num_classes,kernel_size=1)
    self.upsample=nn.Upsample(scale_factor=16, mode='bilinear')

  def forward(self,x):
    x=self.backbone(x)
    x=self.conv_out(x)
    x=self.upsample(x)
    return x
  
  def copy_last_layer(self,fc_out):
    param=fc_out.weight.reshape(7,512,1,1)
    self.conv_out.weight=nn.Parameter(param)
    return 
  

In [30]:

test_input=torch.randn((1,3,224,224))
modelC=VGG11Classification()
out=modelC(test_input)
print(out.shape)
modelS=VGG11Segmentation()
out=modelS(test_input)
print(out.shape)

torch.Size([7])


ModuleAttributeError: 'VGG11Segmentation' object has no attribute 'backbone'

In [31]:
import torch 
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader 
import os 
import cv2 
import numpy as np 
from glob import glob

class MaskDataset(Dataset):
  def __init__(self,data_root,input_size=224, transform=None):
    super(MaskDataset,self).__init__()

    self.img_list=glob(os.path.join(data_root,'*.jpg'))
    self.len=len(self.img_list)
    self.input_size=input_size
    self.transforms= transform 

    def __getitem__(self, index):
      img_path=self.img_list[index]

      # image loading 
      img=cv2.imread(img_path)
      img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      img=img/255. # 이건 이지미의 값들을 전부 255 의 값으로 나눠서 정규화 시킨 모양 
      if self.transform:
        img =self.transform(img )

      # Ground Truth 
      label=self._get_class_idx_from_img_name(img_path)

      return img, label 
    def __len__(self):
      return self.len 

    def _get_class_idx_from_img_name(self,img_path):
      img_name=os.path.basename(img_path)
      




In [33]:
## Model Loading 
model_root ='./model.pth'
modelC=VGG11Classification()
modelC.load_state_dict(torch.load(model_root))
input_size=224
transform=transforms.Compose([
  transforms.ToTensor(),
  transforms.Resize((224,224)),
  transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])
batch_size=1
test_dataset=MaskDataset(data_root,input_size=input_size, transform=transform)
test_loader=DataLoader(test_dataset,batch_size=batch_size,suffle=True, pin_memory=True)


FileNotFoundError: [Errno 2] No such file or directory: './model.pth'

In [ ]:
import matplotlib
modelS.cuda().float()
modeS.eval()

for iter, (img,label) in enumerate(test_loader):
  img=img.flaot().cuda()
  # inference for Semantic Segmentation 
  res=modeS(img)[0]
  heat=res[label[0]]
  resH=

In [ ]:
# import matplotlib.pyplot as plt 
# def show_landmark(image, landmarks):
#   plt.imshow(image)
#   plt.scatter(landmarks[:,0],landmarks[:,1],s=10, marker='.', c='r')
#   plt.pause(0.001)# 갱신이 되도록 잠시 멈춥니다 

# plt.figure()
# show_landmarks(io.imread(os.path.join('data/faces/',img_name)),landmarks)
# plt.show() 

In [ ]:

# class FaceLadnmarksDataset(Dataset):
#   def __init__(self,csv_file,root_dir,transfrom=None):
#     self.landmarks_frame=pd.read_csv(csv_file)
#     self.root_dir=root_dir
#     self.transform=transform 

#   def __len__(self):
#     return len(self.landmarks_frame)

#   def __getitem__(self, idx):
#     if torch.is_tensor(idx):
#       idx=idx.tolist()
    
#     img_name=os.path.join(self.root_dir,self.landmarks_frame.iloc[idx,0])


In [ ]:
# class Rescale(object):

#   def __init__(self,output_size):
#     assert isinstance(output_size, (int,tuple))
#     self.output_size=output_size

#   def __call__(self,sample):
#     image,landmarks=sample['image'],sample['landmarks']

#     h,w = image.shape[:2]

#     if isinstance(self.output_size, int):
#       if h>w: # 높이가 w보다 크다면 h 와 w의 크기를 갖게 맞춰준다 즉 크롭된 영상은 정사각형스타일이다. 
        
